In [18]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import sys
import string

In [2]:
files = ['../data/shakespeare.txt'] 
text = ''

for filename in files:
    with open(filename) as f:
        for line in f:
            line = line.strip()
            if len(line) > 0 and not line.isdigit():
                text += line.lower() + '\n'

In [3]:
# create mapping of unique chars to integers
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [4]:
print(chars)
n_chars = len(text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

['\n', ' ', '!', "'", '(', ')', ',', '-', '.', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Total Characters:  93674
Total Vocab:  38


In [5]:
# prepare the dataset of input to output pairs encoded as binary vectors
seq_length = 40
step = 2
sentences = []
next_chars = []
for i in range(0, n_chars - seq_length, step):
    seq_in = text[i: i + seq_length]
    seq_out = text[i + seq_length]
    sentences.append(seq_in)
    next_chars.append(seq_out)
n_patterns = len(sentences)
print("Total Patterns: ", n_patterns)

Total Patterns:  46817


In [7]:
# binary
X = np.zeros((len(sentences), seq_length, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_to_int[char]] = 1
    y[i, char_to_int[next_chars[i]]] = 1

In [10]:
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(seq_length, len(chars))))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [11]:
def sample(preds, temperature = 1.0):
    # helper function to sample an index from a probability array

    preds = np.asarray(preds).astype('float')
    preds = np.log(preds) / temperature

    # Fix division by 0
    preds[preds == np.inf] = 0

    exp_preds = np.exp(preds)
    preds =  exp_preds / np.sum(exp_preds)
    
    return np.argmax(np.random.multinomial(1, preds, 1))

In [15]:
model.fit(X, y, batch_size=64, epochs=25)
model.save_weights('sonnet_25_64.h5')

Epoch 1/25
46817/46817 [==============================] - 207s 4ms/step - loss: 1.9716
Epoch 2/25
46817/46817 [==============================] - 209s 4ms/step - loss: 1.8200
Epoch 3/25
46817/46817 [==============================] - 199s 4ms/step - loss: 1.7225
Epoch 4/25
46817/46817 [==============================] - 203s 4ms/step - loss: 1.6397
Epoch 5/25
46817/46817 [==============================] - 201s 4ms/step - loss: 1.5719
Epoch 6/25
46817/46817 [==============================] - 202s 4ms/step - loss: 1.5072
Epoch 7/25
46817/46817 [==============================] - 193s 4ms/step - loss: 1.4459
Epoch 8/25
46817/46817 [==============================] - 194s 4ms/step - loss: 1.3770
Epoch 9/25
46817/46817 [==============================] - 199s 4ms/step - loss: 1.3134
Epoch 10/25
46817/46817 [==============================] - 199s 4ms/step - loss: 1.2439
Epoch 11/25
46817/46817 [==============================] - 200s 4ms/step - loss: 1.1735
Epoch 12/25
46817/46817 [================

In [ ]:
model.load_weights('../weights/sonnet_25_64.h5')

In [48]:
char_length = 800  # genrated length
for temperature in [1.5, 0.75, 0.25]:
    print( '\n' + '\n' + 'temperature = ' + str(temperature) + '\n')
    
    generated = 'shall i compare thee to a summer\'s day?\n'
    sentence = generated
    
    #sys.stdout.write(generated)
    for i in range(char_length):
        x = np.zeros((1, seq_length, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_to_int[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = int_to_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char
        
        #sys.stdout.write(next_char)
        #sys.stdout.flush()
    print(generated)



temperature = 1.5

shall i compare thee to a summer's day?
thine for my self, and think to tebpented,
like bainting us a lrevev uwme!it tee.
of be thy gives whed wasch doth mest bud look,
yer when the hote for can of be-tectioned
such awe that well, yet for i leave, devery;
if so rught in dectered, in what should line yees,
even or sext, love hose, my this behild,
when i am sterves to saavy i thre, shall be thy.
and no being boo his and then commonged,
my heavt is thee thrive which is but as motor.
hast i my men my verse hath constrarey,
and such dreadic sanf the pered to keeps,
and after th-all her mourn thus very one,
gov, with from when your proised o's all ,
if is not love's fring my self being ear.
if i in my add of jegjay bage ade?
one.
and that ages in menathing merimr:
for live lose, and fair i have thee lecvesing:
who so recessang will aw


temperature = 0.75

shall i compare thee to a summer's day?
and the perplecure you may i in me tongue,
contion, nor can be black not me 